In [18]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [19]:
#Read main data
data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\data_values_anomaly.xlsx',sheet_name='data_transpose')
rainfall_data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\rainfall_data.xlsx',sheet_name='Sheet1')

# Read data necessary for normalization
ind_type=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\indicator_type_anomaly.xlsx',sheet_name='Sheet1')

In [20]:
#defining normalization function for (dam capicity)/(surface rwr)
def damcap_normalize(ser1,ser2):
    result=ser1*0
    length=ser1.size
    for i in range(length):
        if ser1[i]>=ser2[i]:
            result[i]=(ser1.max()-ser1[i])/(ser1.max()-ser2[i])
        else:
            result[i]=(ser1[i]-ser1.min())/(ser2[i]-ser1.min())
        if result[i]<0.01:
            result[i]=0.01
    return result  

#defining aggregation functions
def aggregate(df,a=0.5):
    size=len(df.columns)
    result=(df.product(axis=1)**(1/size))*a+df.mean(axis=1)*(1-a)
    return result

#defining weighted aggregation functions
def weighted_addminagg(dfi,dfw,a=0.5):
    weight_array=np.array(dfw)[0]
    dfmin=dfi.min(axis=1)
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    result=a*dfmin+(1-a)*weighted_sum
    return result

def weighted_sum(dfi,dfw):
    weight_array=np.array(dfw)[0]
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    return weighted_sum

def geomean(dfi):
    size=len(dfi.columns)
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    return weighted_geomean

def weighted_geomean(dfi,dfw):
    weight_array=np.array(dfw)[0]
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    return weighted_geomean

def weighted_addgeo(dfi,dfw,a=0.5):
    weight_array=np.array(dfw)[0]
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    result= a*weighted_geomean+(1-a)*weighted_sum
    return result

In [21]:
#selecting rainfall data after 1366 (nearly 30 years)
rainfall_modified=rainfall_data.loc[(rainfall_data['syear']>1366) & (rainfall_data['syear']<1399)]

#making a dictionary of provinces and their respective data
pr=list(rainfall_modified.ostan.unique())
prdict={elem:pd.DataFrame() for elem in pr}
for key in prdict.keys():
    prdict[key]=rainfall_modified[:][rainfall_modified.ostan==key]
    
#making a dictionary consisting of province names as keys and annual precipitation as respective values
annualrain_dict={x:pd.DataFrame() for x in pr}
for key in annualrain_dict.keys():
    annualrain_dict[key]=prdict[key].groupby('syear').precnew.sum()

#calculating coefficients of variation
interannual_varicoef={x:pd.DataFrame() for x in pr}
for key in interannual_varicoef.keys():
    interannual_varicoef[key]=annualrain_dict[key].std()/annualrain_dict[key].mean()
monthly_varicoef={x:pd.DataFrame() for x in pr}
for key in monthly_varicoef.keys():
    monthly_varicoef[key]=prdict[key].precnew.std()/prdict[key].precnew.mean()
annualevap_varicoef={x:pd.DataFrame() for x in pr}

#converting monthly coefficient of variation to dataframe and sorting it based on original data
monthlyvaricoeff_df = pd.DataFrame(monthly_varicoef.items(),columns=['province', 'monthly_varicoeff'])
monthlyvaricoeff_df = monthlyvaricoeff_df.drop([31])
monthlyvaricoeff_df = monthlyvaricoeff_df.set_index('province')
monthlyvaricoeff_df = monthlyvaricoeff_df.reindex(index=data['province'])
monthlyvaricoeff_df = monthlyvaricoeff_df.reset_index()

#converting annual coefficient of variation to dataframe and sorting it based on original data
intanvaricoeff_df = pd.DataFrame(interannual_varicoef.items(),columns=['province', 'rain_coeff_variability'])
intanvaricoeff_df = intanvaricoeff_df.drop([31])
intanvaricoeff_df = intanvaricoeff_df.set_index('province')
intanvaricoeff_df = intanvaricoeff_df.reindex(index=data['province'])
intanvaricoeff_df = intanvaricoeff_df.reset_index()

# adding calculated coefficients of variation to the data
data['rain_coeff_variation']=intanvaricoeff_df['rain_coeff_variability']
data['monthly_varicoeff']= monthlyvaricoeff_df['monthly_varicoeff']

In [22]:
#Creating a dataframe consisting of necessary variables for the calculation of sub-indicators
variables=pd.DataFrame()
variables['province']=data.province
variables['irwr']=data.precipitation-data.evaporation
variables['surf_irwr']=(data.precipitation-data.evaporation)*data.runoff_coeff
variables['withdraw_surf']=data.iloc[:,5:8].sum(axis=1)
variables['gw_irwr']=(data.precipitation-data.evaporation)*(1-data.runoff_coeff)
variables['withdraw_gw']=data.iloc[:,8:11].sum(axis=1)
variables['withdraw_agr']=data.withdraw_gw_agr+data.withdraw_surf_agr
variables['withdraw_ind']=data.withdraw_gw_ind+data.withdraw_surf_ind
variables['access_sanitation_total']=(data.access_sanitation_urban*data.urban_pop_ratio+data.access_sanitation_rural*(1-data.urban_pop_ratio))
variables['deficit_gw_annual']=data.annual_gw_variation*(-1)
variables['deficit_gw_aggregate']=data.aggregate_gw_variation*(-1)
variables['withdraw_total']=np.NaN
variables['withdraw_total']=data[['withdraw_surf_agr','withdraw_surf_ind','withdraw_surf_dom','withdraw_gw_agr','withdraw_gw_ind','withdraw_gw_dom']].sum(axis=1)
variables['agr_withdraw_ratio']=variables['withdraw_agr']/variables['withdraw_total']
variables['access_sanitation_total']=data['access_sanitation_urban']*data['urban_pop_ratio']+data['access_sanitation_rural']*(1-data['urban_pop_ratio'])

#setting negative deficit values equal to zero
variables.loc[variables['deficit_gw_annual'] < 0 ,'deficit_gw_annual']=0
variables.loc[variables['deficit_gw_aggregate'] < 0 ,'deficit_gw_aggregate']=0
variables['withdraw_gw_allowable']=variables.gw_irwr-(variables.deficit_gw_aggregate/17)
variables['withdraw_agr_decrease']=variables.withdraw_gw-variables.withdraw_gw_allowable
variables.loc[variables['withdraw_agr_decrease']<0,'withdraw_agr_decrease']=0
variables['agrwat_lost_ratio']=variables.withdraw_agr_decrease/variables.withdraw_agr

In [23]:
# calculating trade-off ratio between modern irrigation and employment lost

water_saved_perhec=variables['withdraw_agr']/(2*data['land_irrig_tot']-data['land_irrig_modern'])

# wwdpmii= water withdrawal decrease per modern irrigation increase (percent)
wwdpmii=water_saved_perhec*0.01*data['land_irrig_tot']
#wdn=water decrease necessary for compensating the effect of lowering 1 percent modern irrigation
wdn=variables['withdraw_agr_decrease']+wwdpmii
#elsftr=employment lost for tradeoff ratio & mitr=modern irrigation for tradeoff ratio
eltr=wdn*data.agr_employment/variables.withdraw_agr
mitr=data.land_irrig_modern/data.land_irrig_tot-0.01
eln=variables['withdraw_agr_decrease']*data.agr_employment/variables.withdraw_agr
tradeoff_ratio=(eltr-eln)*100/18


In [24]:
#Creating a dataframe for the sub-indicators
#Dimension 1: Resources
sub_indicators=pd.DataFrame()
sub_indicators['rain_coeff_variation']=data['rain_coeff_variation']
sub_indicators['monthly_varicoeff']=data['monthly_varicoeff']
sub_indicators['gw_agdef_gwrwr']=variables.deficit_gw_aggregate/variables.gw_irwr
sub_indicators['gw_andef_withdraw']=variables.deficit_gw_annual/variables.withdraw_gw
sub_indicators['agr_dependency_gw']=data.withdraw_gw_agr/variables.withdraw_agr
sub_indicators['ind_dependency_gw']=data.withdraw_gw_ind/variables.withdraw_ind
sub_indicators['urbanwat_dependency_gw']=data.withdraw_urban_gw/data.produced_urban_wat
sub_indicators['ruralwat_dependency_gw']=data.withdraw_rural_gw/data.produced_rural_wat
sub_indicators['withdraw_surf_ratio']=variables.withdraw_surf/variables.surf_irwr
sub_indicators['withdraw_gw_ratio']=variables.withdraw_gw/variables.gw_irwr
sub_indicators['anomaly_rain']=abs(data['anomaly_rain'])
sub_indicators['anomaly_temp']=data['anomaly_temp']
sub_indicators['irwr_percap']=(variables.irwr*(10**6)/data.population)/(1+data['pop_growth']/100)
#Dimension 2: access

sub_indicators['access_wat_urban']=data['access_wat_urban']
sub_indicators['access_wat_rural']=data['access_wat_rural']
sub_indicators['under_stress_pop']=data['under_stress_pop']
sub_indicators['access_sanitation_urban']=data['access_sanitation_urban']
sub_indicators['access_sanitation_rural']=data['access_sanitation_rural']
sub_indicators['treated_municipal_wastewater']=data.waste_facility_cap/data.daily_produced_waste
sub_indicators['quality_proxy']=data.urban_fam_treatwat/data.total_urban_fam
sub_indicators['damcap_rwr_ratio']=data.dam_cap/variables.surf_irwr

#Dimension 3:Economy
sub_indicators['efficiency_agr']=data.agr_added_value/variables.withdraw_agr
sub_indicators['modern_irrig']=data.land_irrig_modern/data.land_irrig_tot
sub_indicators['employment_lost_agr']=variables.agrwat_lost_ratio*data.agr_employment
sub_indicators['unaccounted_wat_urban']=data['unaccounted_wat_urban']
sub_indicators['unaccounted_wat_rural']=data['unaccounted_wat_rural']
sub_indicators['efficiency_ind']=data.ind_added_value/variables.withdraw_ind

sub_indicators.index=variables['province']
sub_indicators.loc['khuz','damcap_rwr_ratio']=1


In [25]:
#Normalizing Data
normalized=sub_indicators*0
max_allowable_withdraw=variables['withdraw_gw_allowable']/variables['gw_irwr']
max_allowable_withdraw.index=sub_indicators.index

for col in ind_type.columns:
    i=ind_type.columns.get_loc(col)
    if ind_type.loc[2,col]=='b': # The bigger the better indicators
        normalized.loc[sub_indicators[col]>=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]<=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]>ind_type.loc[1,col])& (sub_indicators[col]<ind_type.loc[0,col])
        normalized.loc[cond,col]=(sub_indicators.loc[cond,col]-ind_type.loc[1,col])/(ind_type.loc[0,col]-ind_type.loc[1,col])
    elif ind_type.loc[2,col]=='l':      # The lower the better indicators
        normalized.loc[sub_indicators[col]<=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]>=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]<ind_type.loc[1,col])& (sub_indicators[col]>ind_type.loc[0,col])
        normalized.loc[cond,col]=(ind_type.loc[1,col]-sub_indicators.loc[cond,col])/(ind_type.loc[1,col]-ind_type.loc[0,col])
    elif ind_type.loc[2,col]=='diff':    # GW withdrawal to rwr ratio normalization
        cond1=(sub_indicators[col] > max_allowable_withdraw)
        normalized.loc[cond1,col]=0.01
        cond2=sub_indicators[col]<0.25
        normalized.loc[cond2,col]=1
        cond3=(sub_indicators[col] < max_allowable_withdraw) & (sub_indicators[col]>0.25)
        normalized.loc[cond3,col]=(max_allowable_withdraw.loc[cond3]-sub_indicators.loc[cond3,col])/(max_allowable_withdraw.loc[cond3]-0.25)
        

In [26]:
# dam capacity to surface rwr ratio normalization    
variation_coeff_agg=pd.Series(normalized[['rain_coeff_variation','monthly_varicoeff']].mean(axis=1),index=variables['province'])
variation_coeff_bins=pd.cut(variation_coeff_agg,4,labels=['Q1','Q2','Q3','Q4'])
ideal_damcap=pd.Series(index=variables['province'],dtype='float64') 
ideal_damcap[variation_coeff_bins=='Q1']=1
ideal_damcap[variation_coeff_bins=='Q2']=0.9
ideal_damcap[variation_coeff_bins=='Q3']=0.8
ideal_damcap[variation_coeff_bins=='Q4']=0.7
normalized['damcap_rwr_ratio']=damcap_normalize(sub_indicators['damcap_rwr_ratio'],ideal_damcap)   
normalized.loc['khuz','damcap_rwr_ratio']=1

# modifying modern irrigation indicator vlues based on aggregate GW reservoir deficits
gw_deficit_agg=pd.Series(aggregate(normalized[['gw_agdef_gwrwr','gw_andef_withdraw']]),index=variables['province'])
gw_deficit_bins=pd.qcut(gw_deficit_agg,4,labels=['Q1','Q2','Q3','Q4'])
gw_modifier=pd.Series(index=variables['province'],dtype='float64')
gw_modifier[gw_deficit_bins=='Q1']=0.8
gw_modifier[gw_deficit_bins=='Q2']=0.9
gw_modifier[gw_deficit_bins=='Q3']=1
gw_modifier[gw_deficit_bins=='Q4']=1
normalized['modern_irrig']=normalized['modern_irrig'].multiply(gw_modifier)

normalized

,rain_coeff_variation,monthly_varicoeff,gw_agdef_gwrwr,gw_andef_withdraw,agr_dependency_gw,ind_dependency_gw,urbanwat_dependency_gw,ruralwat_dependency_gw,withdraw_surf_ratio,withdraw_gw_ratio,...,access_sanitation_rural,treated_municipal_wastewater,quality_proxy,damcap_rwr_ratio,efficiency_agr,modern_irrig,employment_lost_agr,unaccounted_wat_urban,unaccounted_wat_rural,efficiency_ind
province,,,,,,,,,,,,,,,,,,,,,
azsh,0.830223,0.591822,0.758141,0.835653,0.569003,0.443804,0.541053,0.201991,0.356970,0.028390,...,0.010,0.386507,0.783554,0.416105,0.648512,0.189842,1.000000,0.704000,0.478,0.421995
azgh,0.762235,0.586217,0.755272,0.916886,0.593602,0.525566,0.520602,0.036342,0.385987,0.010000,...,0.075,0.389288,0.941676,0.773622,0.346823,0.160564,0.677908,0.582000,0.634,0.089842
arda,0.910809,0.672874,0.655862,0.550278,0.844037,0.021379,0.750312,0.405109,0.010000,0.370794,...,0.030,0.430248,0.735529,0.580676,0.518322,0.092103,1.000000,0.486667,0.612,0.187797
esfe,0.543145,0.261304,0.010000,0.306784,0.326003,0.705156,0.856976,0.500911,0.261538,0.010000,...,0.040,0.607885,0.812189,0.604238,0.165513,0.262313,0.748706,0.795333,0.608,0.374368
albo,0.732827,0.427278,0.063789,0.557846,0.359067,0.052322,0.233499,0.027027,0.775382,0.010000,...,0.100,0.146851,0.889188,0.999874,0.707274,0.250977,0.914446,0.590000,0.282,0.165413
ilam,0.456402,0.201425,0.938453,1.000000,0.746288,0.450605,0.379525,0.010000,0.640368,0.745842,...,0.010,0.431609,0.734637,0.213246,0.254381,0.372634,1.000000,0.609000,0.596,0.010000
bush,0.041807,0.010000,0.813642,0.448223,0.458492,0.451727,0.841427,0.828221,0.010000,0.040690,...,0.010,0.118561,0.010000,0.636872,0.542424,0.396178,1.000000,0.461667,0.506,0.974587
tehr,0.723369,0.445649,0.010000,0.740442,0.399267,0.120334,0.466937,0.010000,0.010000,0.010000,...,0.120,0.763453,0.727589,0.483971,0.512348,0.270692,0.944761,0.535333,0.450,0.737726
chba,0.645994,0.250813,0.928253,0.876029,0.580888,0.019648,0.017711,0.070671,1.000000,0.749339,...,0.010,0.391572,0.946817,0.512872,0.241696,0.550997,1.000000,0.542000,0.292,0.014486


In [27]:
# reading AHP weights from csv files
eco_weights=pd.read_csv('eco_modified_weights.csv')
acc_weights=pd.read_csv('acc_modified_weights.csv')
res_weights=pd.read_csv('res_modified_weights.csv')
wsi_weights=pd.read_csv('wsi_modified_weights.csv')

eco_weights.drop(columns='Unnamed: 0',inplace=True)
arrayeco=[['E1','E1','E1','E2','E2','E3','E3','E3'],list(eco_weights.iloc[0,:])]
eco_weights.columns=pd.MultiIndex.from_arrays(arrayeco, names=('Aggregation Level', 'Indicator'))
eco_weights.drop(index=0,inplace=True)
eco_weights.index=['Weight']
eco_weights=eco_weights.astype('float')

acc_weights.drop(columns='Unnamed: 0',inplace=True)
arrayacc=[['A1','A1','A1','A2','A2','A2','A3','A3','A3','A3'],list(acc_weights.iloc[0,:])]
acc_weights.columns=pd.MultiIndex.from_arrays(arrayacc, names=('Aggregation Level', 'Indicator'))
acc_weights.drop(index=0,inplace=True)
acc_weights.index=['Weight']
acc_weights=acc_weights.astype('float')

res_weights.drop(columns='Unnamed: 0',inplace=True)
arrayres=[['R1','R1','R2','R2','R3','R3','R3','R3','R4','R4','R5','R5','R5','R6','R6','R7','R7','R7','R7'],list(res_weights.iloc[0,:])]
res_weights.columns=pd.MultiIndex.from_arrays(arrayres, names=('Aggregation Level', 'Indicator'))
res_weights.drop(index=0,inplace=True)
res_weights.index=['Weight']
res_weights=res_weights.astype('float')

wsi_weights.drop(columns='Unnamed: 0',inplace=True)
arraywsi=[['WSI','WSI','WSI'],list(wsi_weights.iloc[0,:])]
wsi_weights.columns=pd.MultiIndex.from_arrays(arraywsi, names=('Aggregation Level', 'Indicator'))
wsi_weights.drop(index=0,inplace=True)
wsi_weights.index=['Weight']
wsi_weights=wsi_weights.astype('float')
ahp_weights=wsi_weights.join(res_weights.join(acc_weights.join(eco_weights,how='outer'),how='outer'),how='outer')

In [28]:
res_sub_columns=['ACV','MCV','Dag','Dan','GWDagr','GWDind','GWDurb','GWDrur','SWS','GWS','APA','ATA','IRWR']
acc_sub_columns=['WAurb','WArur','USP','SAurb','SArur','TPC','WQ','DC']
eco_sub_columns=['AE','MI','EL','NRWurb','NRWrur','IE']
sub_columns1_list= ['R1','R1','R2','R2','R3','R3','R3','R3','R4','R4','R5','R5','R7']+['A1','A1','A1','A2','A2','A2','A3','A3']+['E1','E1','E1','E2','E2','E3']
sub_columns2_list=res_sub_columns+acc_sub_columns+eco_sub_columns
arraycolumns=[sub_columns1_list]+[sub_columns2_list]
normalized_ahp=pd.DataFrame(normalized).round(decimals=4)
normalized_ahp.columns=pd.MultiIndex.from_arrays(arraycolumns, names=('Aggregation Level', 'Indicator'))
normalized_ahp.head()

Aggregation Level      R1              R2              R3                  \
Indicator             ACV     MCV     Dag     Dan  GWDagr  GWDind  GWDurb   
province                                                                    
azsh               0.8302  0.5918  0.7581  0.8357  0.5690  0.4438  0.5411   
azgh               0.7622  0.5862  0.7553  0.9169  0.5936  0.5256  0.5206   
arda               0.9108  0.6729  0.6559  0.5503  0.8440  0.0214  0.7503   
esfe               0.5431  0.2613  0.0100  0.3068  0.3260  0.7052  0.8570   
albo               0.7328  0.4273  0.0638  0.5578  0.3591  0.0523  0.2335   

Aggregation Level              R4          ...     A2              A3          \
Indicator          GWDrur     SWS     GWS  ...  SArur     TPC      WQ      DC   
province                                   ...                                  
azsh               0.2020  0.3570  0.0284  ...  0.010  0.3865  0.7836  0.4161   
azgh               0.0363  0.3860  0.0100  ...  0.075  0.3893  0.9417  0.7736   
arda               0.4051  0.0100  0.3708  ...  0.030  0.4302  0.7355  0.5807   
esfe               0.5009  0.2615  0.0100  ...  0.040  0.6079  0.8122  0.6042   
albo               0.0270  0.7754  0.0100  ...  0.100  0.1469  0.8892  0.9999   

Aggregation Level      E1                      E2             E3  
Indicator              AE      MI      EL  NRWurb NRWrur      IE  
province                                                          
azsh               0.6485  0.1898  1.0000  0.7040  0.478  0.4220  
azgh               0.3468  0.1606  0.6779  0.5820  0.634  0.0898  
arda               0.5183  0.0921  1.0000  0.4867  0.612  0.1878  
esfe               0.1655  0.2623  0.7487  0.7953  0.608  0.3744  
albo               0.7073  0.2510  0.9144  0.5900  0.282  0.1654  

[5 rows x 27 columns]

In [48]:
normalized_lvl2=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
normalized_lvl2.columns=pd.MultiIndex.from_arrays([['R5'],['CV']], names=('Aggregation Level', 'Indicator'))
normalized_lvl2[('R5','CV')]=weighted_sum(normalized_ahp.loc[:,'R1'],ahp_weights['R1']).round(decimals=4)
normalized_lvl2[('R6','GD')]=weighted_sum(normalized_ahp.loc[:,'R2'],ahp_weights['R2']).round(decimals=4)
normalized_lvl2[('R6','GWD')]=weighted_sum(normalized_ahp.loc[:,'R3'],ahp_weights['R3']).round(decimals=4)
normalized_lvl2.head()

Aggregation Level      R5      R6        
Indicator              CV      GD     GWD
province                                 
azsh               0.7658  0.7884  0.4456
azgh               0.7147  0.8183  0.4044
arda               0.8466  0.6147  0.6197
esfe               0.4670  0.1258  0.6071
albo               0.6503  0.2565  0.1938

In [49]:
## Assessing water security using a mix of weighted_sum and minimum aggregation rules

#  calculating indicators 
indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_addminagg(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5']).round(decimals=4)
indicators_weighted[('Resource','WS')]=weighted_addminagg(normalized_ahp.loc[:,'R4'],ahp_weights['R4']).round(decimals=4)
# if a province has low groundwater reservoir deficit then high dependency on groundwater is considered okay i.e. if GD> 0.8 then GWD not involved in aggregation
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_addminagg(normalized_lvl2.loc[:,'R6'],ahp_weights['R6']).round(decimals=4)
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_addminagg(normalized_ahp.loc[:,'A1'],ahp_weights['A1']).round(decimals=4)
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_addminagg(normalized_ahp.loc[:,'A2'],ahp_weights['A2']).round(decimals=4)
indicators_weighted[('Economy','Eagr')]=weighted_addminagg(normalized_ahp.loc[:,'E1'],ahp_weights['E1']).round(decimals=4)
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_addminagg(normalized_ahp.loc[:,'E2'],ahp_weights['E2']).round(decimals=4)

#  calculating dimensions
Dimensions=pd.DataFrame(index=sub_indicators.index)
Dimensions['Resource']=weighted_addminagg(indicators_weighted[('Resource')],ahp_weights['R7']).round(decimals=4)
Dimensions['Access']=weighted_addminagg(indicators_weighted[('Access')],ahp_weights['A3']).round(decimals=4)
Dimensions['Economy']=weighted_addminagg(indicators_weighted[('Economy')],ahp_weights['E3']).round(decimals=4)

#  calculating WSI
WSI=pd.Series(weighted_addminagg(Dimensions,ahp_weights['WSI'])).round(decimals=4)
WSI.sort_values(ascending=False)
# Dimensions.sort_values('Economy',ascending=False)

province
lores    0.3776
mazan    0.3700
kord     0.3610
chba     0.3384
hormo    0.3300
kohbu    0.3261
gilan    0.3240
ilam     0.3231
kersh    0.3148
arda     0.2760
marka    0.2710
azsh     0.2637
azgh     0.2570
fars     0.2565
esfe     0.2263
hame     0.2195
bush     0.2123
albo     0.2121
khosh    0.2080
zanj     0.2020
khuz     0.1945
qazv     0.1851
gole     0.1845
tehr     0.1788
yazd     0.1686
semn     0.1612
kerm     0.1603
khoj     0.1536
sisba    0.1531
qom      0.1482
khor     0.1341
dtype: float64

In [44]:
indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_sum(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5'])
indicators_weighted[('Resource','WS')]=weighted_sum(normalized_ahp.loc[:,'R4'],ahp_weights['R4'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_sum(normalized_lvl2.loc[:,'R6'],ahp_weights['R6'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_sum(normalized_ahp.loc[:,'A1'],ahp_weights['A1'])
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_sum(normalized_ahp.loc[:,'A2'],ahp_weights['A2'])
indicators_weighted[('Economy','Eagr')]=weighted_sum(normalized_ahp.loc[:,'E1'],ahp_weights['E1'])
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_sum(normalized_ahp.loc[:,'E2'],ahp_weights['E2'])

Dimensions=pd.DataFrame(index=sub_indicators.index)
Dimensions['Resource']=weighted_sum(indicators_weighted[('Resource')],ahp_weights['R7'])
Dimensions['Access']=weighted_sum(indicators_weighted[('Access')],ahp_weights['A3'])
Dimensions['Economy']=weighted_sum(indicators_weighted[('Economy')],ahp_weights['E3'])

WSI2=pd.Series(weighted_sum(Dimensions,ahp_weights['WSI']))
# WSI2.sort_values(ascending=False)
Dimensions.sort_values('Resource',ascending=False)

,Resource,Access,Economy
province,,,
gilan,0.944299,0.599712,0.369009
lores,0.927339,0.606491,0.578341
kohbu,0.915260,0.557583,0.483666
chba,0.890999,0.666189,0.400215
ilam,0.858081,0.647227,0.405100
kord,0.857011,0.635465,0.519237
mazan,0.835255,0.532374,0.704616
kersh,0.786809,0.512502,0.404308
khuz,0.585242,0.734676,0.220629


In [32]:
# writer = pd.ExcelWriter('WSI_weighted.xlsx', engine='xlsxwriter')
# WSI.to_excel(writer, sheet_name = 'تجمیع ترکیبی')
# WSI2.to_excel(writer, sheet_name = 'تجمیع حسابی')
# writer.save()

In [33]:
indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_geomean(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5'])
indicators_weighted[('Resource','WS')]=weighted_geomean(normalized_ahp.loc[:,'R4'],ahp_weights['R4'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_geomean(normalized_lvl2.loc[:,'R6'],ahp_weights['R6'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_geomean(normalized_ahp.loc[:,'A1'],ahp_weights['A1'])
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_geomean(normalized_ahp.loc[:,'A2'],ahp_weights['A2'])
indicators_weighted[('Economy','Eagr')]=weighted_geomean(normalized_ahp.loc[:,'E1'],ahp_weights['E1'])
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_geomean(normalized_ahp.loc[:,'E2'],ahp_weights['E2'])

Dimensions=pd.DataFrame(index=sub_indicators.index)
Dimensions['Resource']=weighted_geomean(indicators_weighted[('Resource')],ahp_weights['R7'])
Dimensions['Access']=weighted_geomean(indicators_weighted[('Access')],ahp_weights['A3'])
Dimensions['Economy']=weighted_geomean(indicators_weighted[('Economy')],ahp_weights['E3'])

WSI3=pd.Series(weighted_geomean(Dimensions,ahp_weights['WSI']))
WSI3.sort_values(ascending=False)

province
chba     0.597741
gilan    0.592574
kord     0.588501
lores    0.583810
mazan    0.579968
kohbu    0.569488
ilam     0.559847
kersh    0.487109
arda     0.393013
azsh     0.373201
azgh     0.357243
khuz     0.328520
hormo    0.327813
fars     0.300181
marka    0.293701
esfe     0.292233
zanj     0.274990
hame     0.264663
albo     0.258349
gole     0.258325
bush     0.254528
khosh    0.241005
khoj     0.217479
sisba    0.210076
qazv     0.208645
semn     0.198470
tehr     0.196502
yazd     0.196294
kerm     0.194131
qom      0.173914
khor     0.166108
dtype: float64

In [ ]:
indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_addgeo(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5'])
indicators_weighted[('Resource','WS')]=weighted_addgeo(normalized_ahp.loc[:,'R4'],ahp_weights['R4'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_addgeo(normalized_lvl2.loc[:,'R6'],ahp_weights['R6'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_addgeo(normalized_ahp.loc[:,'A1'],ahp_weights['A1'])
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_addgeo(normalized_ahp.loc[:,'A2'],ahp_weights['A2'])
indicators_weighted[('Economy','Eagr')]=weighted_addgeo(normalized_ahp.loc[:,'E1'],ahp_weights['E1'])
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_addgeo(normalized_ahp.loc[:,'E2'],ahp_weights['E2'])

Dimensions=pd.DataFrame(index=sub_indicators.index)
Dimensions['Resource']=weighted_addgeo(indicators_weighted[('Resource')],ahp_weights['R7'])
Dimensions['Access']=weighted_addgeo(indicators_weighted[('Access')],ahp_weights['A3'])
Dimensions['Economy']=weighted_addgeo(indicators_weighted[('Economy')],ahp_weights['E3'])

WSI3=pd.Series(weighted_addgeo(Dimensions,ahp_weights['WSI']))
WSI3.sort_values(ascending=False)